In [9]:
import pandas as pd

df = pd.read_csv('../data/03_casos/df_53_cleaned.csv')

In [10]:
df.head()

,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt,...,defunciones_60mas_lag_3_semana,defunciones_60mas_lag_4_semana,defunciones_60mas_lag_5_semana,defunciones_60mas_lag_6_semana,mes,bimestre,trimestre,cuatrimestre,semestre,pandemia_covid
0,2000,1,2660,14,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
1,2000,2,2567,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
2,2000,3,2230,15,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
3,2000,4,2067,12,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
4,2000,5,1677,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,2,1,1,1,1,0


In [11]:
df.columns

Index(['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
       'hospitalizados_men5', 'hospitalizados_60mas', 'defunciones_men5',
       'defunciones_60mas', 'sub_reg_nt', 'ira_no_neumonia_lag_1_semana',
       'ira_no_neumonia_lag_2_semana', 'ira_no_neumonia_lag_3_semana',
       'ira_no_neumonia_lag_4_semana', 'ira_no_neumonia_lag_5_semana',
       'ira_no_neumonia_lag_6_semana', 'neumonias_men5_lag_1_semana',
       'neumonias_men5_lag_2_semana', 'neumonias_men5_lag_3_semana',
       'neumonias_men5_lag_4_semana', 'neumonias_men5_lag_5_semana',
       'neumonias_men5_lag_6_semana', 'neumonias_60mas_lag_1_semana',
       'neumonias_60mas_lag_2_semana', 'neumonias_60mas_lag_3_semana',
       'neumonias_60mas_lag_4_semana', 'neumonias_60mas_lag_5_semana',
       'neumonias_60mas_lag_6_semana', 'hospitalizados_men5_lag_1_semana',
       'hospitalizados_men5_lag_2_semana', 'hospitalizados_men5_lag_3_semana',
       'hospitalizados_men5_lag_4_semana', 'hospitalizad

Vamos a entrenar un mismo modelo de forecasting para la predicción de `hospitalizados_men5`, considerando los siguientes escenarios:

### Por años [2000 a 2023], [2001 a 2023], [2002 a 2023], [2003 a 2023], [2004 a 2023],[2005 a 2023], [2006 a 2023], [2007 a 2023], [2008 a 2023], [2009 a 2023], [2010 a 2023], [2011 a 2023], [2012 a 2023], [2013 a 2023], [2014 a 2023], [2015 a 2023], [2016 a 2023], [2017 a 2023], [2018 a 2023], [2019 a 2023], [2020 a 2023], [2021 a 2023], [2022 a 2023]


- Entrenar usando:  
  **Variables:** `ano`, `semana`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `pandemia_covid`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`  
  **Target:** `hospitalizados_men5`

  - Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`  ,  `pandemia_covid`  
  **Target:** `hospitalizados_men5`



- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`, y sus variables *lag* (`_lag_1_semana` a `_lag_6_semana`)  ,  `pandemia_covid` 
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`, y sus variables *lag* (`_lag_1_semana` a `_lag_6_semana`)  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `hospitalizados_60mas_lag_1_semana`, `defunciones_men5_lag_1_semana`, `defunciones_60mas_lag_1_semana`, `defunciones_men5_lag_3_semana`   
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `hospitalizados_60mas_lag_1_semana`, `defunciones_men5_lag_1_semana`, `defunciones_60mas_lag_1_semana`, `defunciones_men5_lag_3_semana`   , `pandemia_covid`  
  **Target:** `hospitalizados_men5`



In [12]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURACIÓN AVANZADA DE CATBOOST PARA MÚLTIPLES ESCENARIOS
# ============================================================================

class CatBoostAdvancedModel:
    """
    Clase para entrenamiento avanzado de CatBoost con:
    - Hyperparameter tuning con Optimización Bayesiana
    - Time Series Cross-Validation
    - Soporte para múltiples periodos y conjuntos de variables
    """
    
    def __init__(self, target_col='hospitalizados_men5', random_state=42):
        self.target_col = target_col
        self.random_state = random_state
        self.best_model = None
        self.best_params = None
        
    def prepare_data(self, df, exog_vars, train_size=0.8):
        """Preparar datos para entrenamiento"""
        df_sorted = df.sort_values(['ano', 'semana']).reset_index(drop=True)
        
        X = df_sorted[exog_vars].copy()
        y = df_sorted[self.target_col].copy()
        
        split_idx = int(len(y) * train_size)
        
        X_train = X[:split_idx]
        X_test = X[split_idx:]
        y_train = y[:split_idx]
        y_test = y[split_idx:]
        
        return X_train, X_test, y_train, y_test
    
    def bayesian_optimization(self, X_train, y_train, n_trials=50):
        """
        Optimización bayesiana de hiperparámetros
        """
        try:
            import optuna
            optuna.logging.set_verbosity(optuna.logging.WARNING)
            
            def objective(trial):
                params = {
                    'iterations': trial.suggest_int('iterations', 300, 1500),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                    'depth': trial.suggest_int('depth', 4, 10),
                    'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
                    'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 30),
                    'random_strength': trial.suggest_float('random_strength', 1, 10),
                    'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
                    'border_count': trial.suggest_int('border_count', 32, 255),
                    'loss_function': 'RMSE',
                    'verbose': False,
                    'random_state': self.random_state,
                    'allow_writing_files': False
                }
                
                # Cross-validation
                tscv = TimeSeriesSplit(n_splits=3)
                scores = []
                
                for train_idx, val_idx in tscv.split(X_train):
                    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                    
                    model = CatBoostRegressor(**params)
                    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=False)
                    
                    pred = model.predict(X_val)
                    mae = mean_absolute_error(y_val, pred)
                    scores.append(mae)
                
                return np.mean(scores)
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
            
            self.best_params = study.best_params
            return self.best_params
            
        except ImportError:
            # Si no está instalado Optuna, usar parámetros por defecto optimizados
            self.best_params = {
                'iterations': 1000,
                'learning_rate': 0.05,
                'depth': 6,
                'l2_leaf_reg': 3,
                'min_data_in_leaf': 10,
                'random_strength': 2,
                'bagging_temperature': 0.5,
                'border_count': 128,
                'loss_function': 'RMSE',
                'verbose': False,
                'random_state': self.random_state,
                'allow_writing_files': False
            }
            return self.best_params
    
    def train_and_evaluate(self, X_train, y_train, X_test, y_test, params=None):
        """
        Entrenar modelo y retornar métricas
        """
        if params is None:
            params = self.best_params
        
        # Crear pools
        train_pool = Pool(X_train, y_train)
        test_pool = Pool(X_test, y_test)
        
        # Entrenar modelo
        self.best_model = CatBoostRegressor(**params)
        self.best_model.fit(train_pool, eval_set=test_pool, verbose=False)
        
        # Predicciones
        y_pred = self.best_model.predict(X_test)
        
        # Métricas
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        return mae, rmse, r2

print("✅ Clase CatBoostAdvancedModel cargada exitosamente")

✅ Clase CatBoostAdvancedModel cargada exitosamente


In [13]:
# Diccionario para almacenar resultados
resultados_modelos = {}

In [14]:
# Definir conjuntos de variables según las especificaciones
variables_sets = {
    'set1_ano_semana': ['ano', 'semana'],
    
    'set2_con_pandemia': ['ano', 'semana', 'pandemia_covid'],
    
    'set3_principales': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                         'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas'],
    
    'set4_principales_con_pandemia': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                                      'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas', 
                                      'pandemia_covid'],
    
    'set5_con_lags_pandemia': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                               'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas', 
                               'pandemia_covid'],
    
    'set6_con_lags': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                      'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas'],
    
    'set7_lags_selectos': ['ano', 'semana', 'hospitalizados_60mas_lag_1_semana',
                           'defunciones_men5_lag_1_semana', 'defunciones_60mas_lag_1_semana',
                           'defunciones_men5_lag_3_semana'],
    
    'set8_lags_selectos_pandemia': ['ano', 'semana', 'hospitalizados_60mas_lag_1_semana',
                                    'defunciones_men5_lag_1_semana', 'defunciones_60mas_lag_1_semana',
                                    'defunciones_men5_lag_3_semana', 'pandemia_covid']
}

# Agregar variables lag a set5 y set6
lag_vars = []
for var in ['ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
            'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas']:
    for i in range(1, 7):
        lag_vars.append(f'{var}_lag_{i}_semana')

variables_sets['set5_con_lags_pandemia'].extend(lag_vars)
variables_sets['set6_con_lags'].extend(lag_vars)

print("Conjuntos de variables definidos:")
for key, vars_list in variables_sets.items():
    print(f"  {key}: {len(vars_list)} variables")

Conjuntos de variables definidos:
  set1_ano_semana: 2 variables
  set2_con_pandemia: 3 variables
  set3_principales: 8 variables
  set4_principales_con_pandemia: 9 variables
  set5_con_lags_pandemia: 45 variables
  set6_con_lags: 44 variables
  set7_lags_selectos: 6 variables
  set8_lags_selectos_pandemia: 7 variables


In [15]:
# Definir todos los periodos de años a evaluar
periodos_anos = [
    (2000, 2023), (2001, 2023), (2002, 2023), (2003, 2023), (2004, 2023),
    (2005, 2023), (2006, 2023), (2007, 2023), (2008, 2023), (2009, 2023),
    (2010, 2023), (2011, 2023), (2012, 2023), (2013, 2023), (2014, 2023),
    (2015, 2023), (2016, 2023), (2017, 2023), (2018, 2023), (2019, 2023),
    (2020, 2023), (2021, 2023), (2022, 2023)
]

print(f"Total de periodos a evaluar: {len(periodos_anos)}")
print(f"Desde [{periodos_anos[0][0]}-{periodos_anos[0][1]}] hasta [{periodos_anos[-1][0]}-{periodos_anos[-1][1]}]")

Total de periodos a evaluar: 23
Desde [2000-2023] hasta [2022-2023]


### Entrenamiento de CatBoost Avanzado con Optimización Bayesiana

Entrenaremos modelos CatBoost con configuración avanzada para cada combinación de periodo y conjunto de variables.

In [16]:
# Configuración
N_TRIALS_OPTUNA = 30  # Número de pruebas para optimización bayesiana (ajustar según tiempo disponible)
USE_OPTIMIZATION = True  # True para optimizar hiperparámetros, False para usar config. por defecto

total_modelos = len(periodos_anos) * len(variables_sets)
contador = 0

print(f"{'='*100}")
print(f"INICIANDO ENTRENAMIENTO DE CATBOOST AVANZADO")
print(f"{'='*100}")
print(f"\nTotal de modelos a entrenar: {total_modelos}")
print(f"Optimización Bayesiana: {'ACTIVADA' if USE_OPTIMIZATION else 'DESACTIVADA'}")
if USE_OPTIMIZATION:
    print(f"Trials por modelo: {N_TRIALS_OPTUNA}")
print(f"\n{'='*100}\n")

for ano_inicio, ano_fin in periodos_anos:
    periodo_label = f"{ano_inicio}-{ano_fin}"
    
    # Preparar datos para el periodo actual
    df_periodo = df[(df['ano'] >= ano_inicio) & (df['ano'] <= ano_fin)].copy()
    df_periodo = df_periodo.sort_values(['ano', 'semana']).reset_index(drop=True)
    
    # Verificar que hay suficientes datos
    if len(df_periodo) < 20:
        print(f"⚠ Periodo {periodo_label}: Datos insuficientes ({len(df_periodo)} obs), omitiendo...")
        continue
    
    print(f"\n{'─'*100}")
    print(f"PERIODO: [{periodo_label}] - {len(df_periodo)} observaciones")
    print(f"{'─'*100}")
    
    for set_name, exog_vars in variables_sets.items():
        contador += 1
        
        # Verificar que todas las variables existen en el dataframe
        vars_disponibles = [v for v in exog_vars if v in df_periodo.columns]
        if len(vars_disponibles) != len(exog_vars):
            vars_faltantes = set(exog_vars) - set(vars_disponibles)
            print(f"  [{contador}/{total_modelos}] ✗ {set_name}: Variables faltantes {vars_faltantes}")
            continue
        
        # Nombre descriptivo del conjunto de variables
        var_desc_map = {
            'set1_ano_semana': 'ano, semana',
            'set2_con_pandemia': 'ano, semana, pandemia_covid',
            'set3_principales': 'principales',
            'set4_principales_con_pandemia': 'principales + pandemia_covid',
            'set5_con_lags_pandemia': 'principales + lags + pandemia_covid',
            'set6_con_lags': 'principales + lags',
            'set7_lags_selectos': 'lags selectos',
            'set8_lags_selectos_pandemia': 'lags selectos + pandemia_covid'
        }
        var_desc = var_desc_map.get(set_name, set_name)
        
        try:
            # Crear instancia del modelo avanzado
            model_manager = CatBoostAdvancedModel(target_col='hospitalizados_men5', random_state=42)
            
            # Preparar datos
            X_train, X_test, y_train, y_test = model_manager.prepare_data(
                df_periodo, exog_vars, train_size=0.8
            )
            
            # Optimización de hiperparámetros (si está activada)
            if USE_OPTIMIZATION:
                best_params = model_manager.bayesian_optimization(X_train, y_train, n_trials=N_TRIALS_OPTUNA)
            else:
                # Usar configuración por defecto optimizada
                best_params = {
                    'iterations': 1000,
                    'learning_rate': 0.05,
                    'depth': 6,
                    'l2_leaf_reg': 3,
                    'min_data_in_leaf': 10,
                    'random_strength': 2,
                    'bagging_temperature': 0.5,
                    'border_count': 128,
                    'loss_function': 'RMSE',
                    'verbose': False,
                    'random_state': 42,
                    'allow_writing_files': False
                }
                model_manager.best_params = best_params
            
            # Entrenar y evaluar
            mae, rmse, r2 = model_manager.train_and_evaluate(X_train, y_train, X_test, y_test)
            
            # Guardar resultados
            resultados_modelos[f'CatBoost-Advanced_{periodo_label}_{set_name}'] = {
                'periodo': periodo_label,
                'modelo': 'CatBoost-Advanced',
                'variables': var_desc,
                'MAE': mae,
                'RMSE': rmse,
                'R2': r2
            }
            
            print(f"  [{contador}/{total_modelos}] ✓ {set_name:30} | MAE: {mae:6.3f} | RMSE: {rmse:6.3f} | R²: {r2:7.4f}")
            
        except Exception as e:
            print(f"  [{contador}/{total_modelos}] ✗ Error en {set_name}: {str(e)[:80]}")

print(f"\n{'='*100}")
print(f"ENTRENAMIENTO COMPLETADO")
print(f"Total de modelos entrenados exitosamente: {len(resultados_modelos)}")
print(f"{'='*100}")

INICIANDO ENTRENAMIENTO DE CATBOOST AVANZADO

Total de modelos a entrenar: 184
Optimización Bayesiana: ACTIVADA
Trials por modelo: 30



────────────────────────────────────────────────────────────────────────────────────────────────────
PERIODO: [2000-2023] - 1248 observaciones
────────────────────────────────────────────────────────────────────────────────────────────────────
  [1/184] ✓ set1_ano_semana                | MAE:  3.329 | RMSE:  3.948 | R²: -0.0577
  [2/184] ✓ set2_con_pandemia              | MAE:  3.394 | RMSE:  3.980 | R²: -0.0751
  [3/184] ✓ set3_principales               | MAE:  2.659 | RMSE:  3.164 | R²:  0.3206
  [4/184] ✓ set4_principales_con_pandemia  | MAE:  2.867 | RMSE:  3.376 | R²:  0.2268
  [5/184] ✓ set5_con_lags_pandemia         | MAE:  2.555 | RMSE:  3.071 | R²:  0.3600
  [6/184] ✓ set6_con_lags                  | MAE:  2.584 | RMSE:  3.131 | R²:  0.3350
  [7/184] ✓ set7_lags_selectos             | MAE:  3.125 | RMSE:  3.771 | R²:  0.0351
  [8/184] ✓ set8_

---

## Tabla Comparativa y Exportación de Resultados

In [17]:
# Crear DataFrame con todos los resultados
df_resultados = pd.DataFrame(resultados_modelos).T
df_resultados = df_resultados.reset_index(drop=True)

# Redondear métricas
df_resultados[['MAE', 'RMSE', 'R2']] = df_resultados[['MAE', 'RMSE', 'R2']].round(4)

# Ordenar resultados por periodo y variables
df_resultados = df_resultados.sort_values(['periodo', 'variables']).reset_index(drop=True)

print("="*120)
print("TABLA COMPARATIVA - CATBOOST AVANZADO CON OPTIMIZACIÓN BAYESIANA")
print("="*120)
print("\nTarget: hospitalizados_men5")
print("\nFormato: periodo, modelo, variables, MAE, RMSE, R2")
print("\nConfiguración:")
print(f"  - Optimización: {'Bayesiana (Optuna)' if USE_OPTIMIZATION else 'Parámetros por defecto'}")
if USE_OPTIMIZATION:
    print(f"  - Trials: {N_TRIALS_OPTUNA}")
print("  - Time Series Cross-Validation: 3 folds")
print("  - Métricas optimizadas: MAE")
print("\nColumnas:")
print("  - Periodo: Rango de años utilizado")
print("  - Modelo: CatBoost-Advanced (con hiperparámetros optimizados)")
print("  - Variables: Conjunto de variables utilizadas")
print("  - MAE: Mean Absolute Error (menor es mejor)")
print("  - RMSE: Root Mean Squared Error (menor es mejor)")
print("  - R²: Coeficiente de Determinación (mayor es mejor, cercano a 1)")
print("\n" + "="*120)

# Mostrar primeras filas
print("\nPrimeros 20 resultados:")
print(df_resultados.head(20).to_string(index=True))
print("\n" + "="*120)

TABLA COMPARATIVA - CATBOOST AVANZADO CON OPTIMIZACIÓN BAYESIANA

Target: hospitalizados_men5

Formato: periodo, modelo, variables, MAE, RMSE, R2

Configuración:
  - Optimización: Bayesiana (Optuna)
  - Trials: 30
  - Time Series Cross-Validation: 3 folds
  - Métricas optimizadas: MAE

Columnas:
  - Periodo: Rango de años utilizado
  - Modelo: CatBoost-Advanced (con hiperparámetros optimizados)
  - Variables: Conjunto de variables utilizadas
  - MAE: Mean Absolute Error (menor es mejor)
  - RMSE: Root Mean Squared Error (menor es mejor)
  - R²: Coeficiente de Determinación (mayor es mejor, cercano a 1)


Primeros 20 resultados:
      periodo             modelo                            variables       MAE      RMSE        R2
0   2000-2023  CatBoost-Advanced                          ano, semana  3.329126  3.948107 -0.057717
1   2000-2023  CatBoost-Advanced          ano, semana, pandemia_covid  3.394196  3.980462 -0.075124
2   2000-2023  CatBoost-Advanced                        lags sel

### Exportar Resultados a CSV

In [18]:
# Crear directorio de resultados si no existe
import os
output_dir = '../data/05_hyper_resultados_modelos'
os.makedirs(output_dir, exist_ok=True)

# Guardar resultados completos en CSV con formato: periodo,modelo,variables,MAE,RMSE,R2
output_path_full = f'{output_dir}/resultados_catboost_advanced_hospitalizados_men5_completo.csv'
df_resultados.to_csv(output_path_full, index=False)
print(f"✓ Resultados completos guardados en: {output_path_full}")
print(f"  Formato: periodo, modelo, variables, MAE, RMSE, R2")

# Guardar resumen por periodo
output_path_resumen = f'{output_dir}/resultados_catboost_advanced_hospitalizados_men5_resumen.csv'
df_resumen = df_resultados.groupby('periodo').agg({
    'MAE': ['mean', 'std', 'min', 'max'],
    'RMSE': ['mean', 'std', 'min', 'max'],
    'R2': ['mean', 'std', 'min', 'max']
}).round(4)
df_resumen.to_csv(output_path_resumen)
print(f"✓ Resumen por periodo guardado en: {output_path_resumen}")

# Guardar top modelos ordenados por R²
output_path_top = f'{output_dir}/resultados_catboost_advanced_hospitalizados_men5_top10.csv'
df_top = df_resultados.nlargest(10, 'R2')[['periodo', 'modelo', 'variables', 'MAE', 'RMSE', 'R2']]
df_top.to_csv(output_path_top, index=False)
print(f"✓ Top 10 modelos guardados en: {output_path_top}")

# Mejores modelos
mejor_mae = df_resultados.loc[df_resultados['MAE'].idxmin()]
mejor_rmse = df_resultados.loc[df_resultados['RMSE'].idxmin()]
mejor_r2 = df_resultados.loc[df_resultados['R2'].idxmax()]

# Resumen final
print("\n" + "="*120)
print("RESUMEN FINAL")
print("="*120)
print(f"\nTotal de modelos entrenados: {len(df_resultados)}")
print(f"Periodos analizados: {df_resultados['periodo'].nunique()}")
print(f"Conjuntos de variables: {df_resultados['variables'].nunique()}")
print(f"  Variables disponibles: {list(df_resultados['variables'].unique())}")

print(f"\nModelo utilizado: CatBoost-Advanced")
print(f"  - Optimización: {'Bayesiana con Optuna' if USE_OPTIMIZATION else 'Parámetros por defecto'}")
if USE_OPTIMIZATION:
    print(f"  - Trials por modelo: {N_TRIALS_OPTUNA}")
print(f"  - Time Series CV: 3 folds")
print(f"  - Métrica optimizada: MAE")

print(f"\nTarget: hospitalizados_men5")

print(f"\n{'─'*120}")
print("MEJORES MODELOS:")
print(f"{'─'*120}")

print(f"\nMenor MAE: {mejor_mae['MAE']:.4f}")
print(f"  Periodo: {mejor_mae['periodo']}")
print(f"  Variables: {mejor_mae['variables']}")
print(f"  RMSE: {mejor_mae['RMSE']:.4f}")
print(f"  R²: {mejor_mae['R2']:.4f}")

print(f"\nMenor RMSE: {mejor_rmse['RMSE']:.4f}")
print(f"  Periodo: {mejor_rmse['periodo']}")
print(f"  Variables: {mejor_rmse['variables']}")
print(f"  MAE: {mejor_rmse['MAE']:.4f}")
print(f"  R²: {mejor_rmse['R2']:.4f}")

print(f"\nMayor R²: {mejor_r2['R2']:.4f}")
print(f"  Periodo: {mejor_r2['periodo']}")
print(f"  Variables: {mejor_r2['variables']}")
print(f"  MAE: {mejor_r2['MAE']:.4f}")
print(f"  RMSE: {mejor_r2['RMSE']:.4f}")

print("\n" + "="*120)
print("Archivos de resultados generados exitosamente.")
print("="*120)

✓ Resultados completos guardados en: ../data/05_hyper_resultados_modelos/resultados_catboost_advanced_hospitalizados_men5_completo.csv
  Formato: periodo, modelo, variables, MAE, RMSE, R2
✓ Resumen por periodo guardado en: ../data/05_hyper_resultados_modelos/resultados_catboost_advanced_hospitalizados_men5_resumen.csv


TypeError: Column 'R2' has dtype object, cannot use method 'nlargest' with this dtype